# Integrantes:
- Carlos Valladares 221164
- Brandon Reyes 22992

# Tarea 1

Parte 1: Carga del dataset MNIST

In [1]:
# Importar las bibliotecas necesarias
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Paso 1: Definir una transformación para las imágenes
# Convertimos las imágenes a tensores y las normalizamos
# La normalización utiliza la media y desviación estándar de los datos MNIST
transformacion_datos = transforms.Compose([
    transforms.ToTensor(),  # Convierte la imagen en tensor (de PIL a tensor)
    transforms.Normalize((0.1307,), (0.3081,))  # Normalización usando media y desviación estándar de MNIST
])

# Paso 2: Descargar y cargar el dataset MNIST para entrenamiento
# train=True indica que queremos el conjunto de entrenamiento
# download=True descarga los datos si no están presentes
dataset_entrenamiento = torchvision.datasets.MNIST(
    root='./datos_mnist',
    train=True,
    transform=transformacion_datos,
    download=True
)

# Paso 3: Descargar y cargar el dataset MNIST para prueba
# train=False indica que es el conjunto de prueba
dataset_prueba = torchvision.datasets.MNIST(
    root='./datos_mnist',
    train=False,
    transform=transformacion_datos,
    download=True
)

# Paso 4: Crear los cargadores de datos (DataLoader)
# Esto nos permite manejar los datos por lotes (batch)
# shuffle=True para mezclar los datos en cada época (solo en entrenamiento)
cargador_entrenamiento = DataLoader(
    dataset=dataset_entrenamiento,
    batch_size=64,
    shuffle=True
)

cargador_prueba = DataLoader(
    dataset=dataset_prueba,
    batch_size=1000,
    shuffle=False
)


Parte 2: Modelo MLP básico (Multilayer Perceptron)

In [2]:
# Importar las bibliotecas necesarias
import torch
import torch.nn as nn
import torch.nn.functional as F

# Definir la clase para el modelo MLP
class ModeloMLPBasico(nn.Module):
    """
    Esta clase representa un Perceptrón Multicapa (MLP) simple con:
    - Una capa de entrada de tamaño 784 (28x28 píxeles aplanados)
    - Dos capas ocultas con funciones de activación ReLU
    - Una capa de salida con 10 neuronas (una por clase del 0 al 9)
    """

    def __init__(self):
        # Llamamos al constructor de la clase base nn.Module
        super(ModeloMLPBasico, self).__init__()

        # Primera capa oculta: de 784 entradas a 128 neuronas
        self.capa_oculta1 = nn.Linear(784, 128)

        # Segunda capa oculta: de 128 a 64 neuronas
        self.capa_oculta2 = nn.Linear(128, 64)

        # Capa de salida: de 64 a 10 neuronas (una por clase)
        self.capa_salida = nn.Linear(64, 10)

    def forward(self, imagen):
        """
        Función de propagación hacia adelante.
        Convierte la imagen de entrada en un vector y pasa por las capas ocultas con activación ReLU.
        Devuelve los logits de salida sin aplicar softmax (porque se usará CrossEntropyLoss).
        """

        # Aplanar la imagen de 28x28 píxeles a un vector de 784
        imagen_aplanada = imagen.view(-1, 784)

        # Aplicar la primera capa oculta y función ReLU
        salida_capa1 = self.capa_oculta1(imagen_aplanada)
        activacion1 = F.relu(salida_capa1)

        # Aplicar la segunda capa oculta y función ReLU
        salida_capa2 = self.capa_oculta2(activacion1)
        activacion2 = F.relu(salida_capa2)

        # Aplicar la capa de salida (no se aplica softmax aquí)
        salida = self.capa_salida(activacion2)

        return salida


In [3]:
# Crear una instancia del modelo MLP básico
modelo_basico = ModeloMLPBasico()


Parte 3: Entrenamiento y evaluación del modelo

In [4]:
# Importar las bibliotecas necesarias para entrenamiento
import torch.optim as optim

# Definir función de entrenamiento del modelo
def entrenar_modelo(modelo, cargador_entrenamiento, cantidad_epocas, tasa_aprendizaje):
    """
    Esta función entrena un modelo MLP utilizando:
    - Un optimizador (Adam)
    - Una función de pérdida (CrossEntropyLoss)
    - Datos del conjunto de entrenamiento por lotes

    Parámetros:
    - modelo: instancia del modelo MLP a entrenar
    - cargador_entrenamiento: DataLoader con los datos de entrenamiento
    - cantidad_epocas: número de veces que se recorrerán todos los datos
    - tasa_aprendizaje: learning rate para el optimizador
    """

    # Definir la función de pérdida (para clasificación multiclase)
    funcion_perdida = nn.CrossEntropyLoss()

    # Definir el optimizador (Adam es una buena opción inicial)
    optimizador = optim.Adam(modelo.parameters(), lr=tasa_aprendizaje)

    # Bucle de entrenamiento por cada época
    for epoca in range(cantidad_epocas):
        print("Época número " + str(epoca + 1) + " iniciada.")
        modelo.train()  # Cambiar el modelo a modo de entrenamiento

        # Recorrer todos los lotes de entrenamiento
        for imagenes, etiquetas in cargador_entrenamiento:
            # Propagación hacia adelante (forward pass)
            salidas = modelo(imagenes)
            perdida = funcion_perdida(salidas, etiquetas)

            # Borrado del gradiente anterior
            optimizador.zero_grad()

            # Retropropagación (cálculo del gradiente)
            perdida.backward()

            # Actualización de los pesos
            optimizador.step()

        print("Época número " + str(epoca + 1) + " finalizada.\n")


In [5]:
def evaluar_modelo(modelo, cargador_prueba):
    """
    Esta función evalúa la precisión del modelo utilizando el conjunto de prueba.

    Parámetros:
    - modelo: modelo MLP ya entrenado
    - cargador_prueba: DataLoader con los datos de prueba

    Devuelve:
    - precisión (% de aciertos)
    """

    modelo.eval()  # Cambiar el modelo a modo de evaluación

    cantidad_correctos = 0
    cantidad_total = 0

    # No se calculan gradientes durante la evaluación
    with torch.no_grad():
        for imagenes, etiquetas in cargador_prueba:
            salidas = modelo(imagenes)
            _, predicciones = torch.max(salidas.data, 1)  # Obtener la clase con mayor valor

            cantidad_total += etiquetas.size(0)
            cantidad_correctos += (predicciones == etiquetas).sum().item()

    precision = 100 * cantidad_correctos / cantidad_total
    print("Precisión del modelo en el conjunto de prueba: " + str(precision) + " %")
    return precision


Parte 4.1 – Modelo 2: Más profundo, activación Tanh


Ejecución del entrenamiento y evaluación

In [7]:
# Crear instancia del modelo
modelo_basico = ModeloMLPBasico()

# Entrenar el modelo (por ejemplo, con 10 épocas y learning rate 0.01)
entrenar_modelo(modelo_basico, cargador_entrenamiento, cantidad_epocas=10, tasa_aprendizaje=0.01)

# Evaluar el modelo
precision_final = evaluar_modelo(modelo_basico, cargador_prueba)


Época número 1 iniciada.
Época número 1 finalizada.

Época número 2 iniciada.
Época número 2 finalizada.

Época número 3 iniciada.
Época número 3 finalizada.

Época número 4 iniciada.
Época número 4 finalizada.

Época número 5 iniciada.
Época número 5 finalizada.

Época número 6 iniciada.
Época número 6 finalizada.

Época número 7 iniciada.
Época número 7 finalizada.

Época número 8 iniciada.
Época número 8 finalizada.

Época número 9 iniciada.
Época número 9 finalizada.

Época número 10 iniciada.
Época número 10 finalizada.

Precisión del modelo en el conjunto de prueba: 96.09 %


Modelo 2: Más profundo, activación Tanh

In [8]:
class ModeloMLPProfundo(nn.Module):
    """
    Modelo MLP con:
    - Tres capas ocultas: 256, 128, 64 neuronas respectivamente
    - Función de activación Tanh en cada capa
    - Capa de salida con 10 neuronas
    """
    def __init__(self):
        super(ModeloMLPProfundo, self).__init__()

        # Primera capa: entrada 784 → capa oculta 1 (256 neuronas)
        self.capa_oculta1 = nn.Linear(784, 256)

        # Segunda capa: capa oculta 1 → capa oculta 2 (128 neuronas)
        self.capa_oculta2 = nn.Linear(256, 128)

        # Tercera capa: capa oculta 2 → capa oculta 3 (64 neuronas)
        self.capa_oculta3 = nn.Linear(128, 64)

        # Capa de salida: 64 neuronas → 10 clases
        self.capa_salida = nn.Linear(64, 10)

    def forward(self, imagen):
        imagen_aplanada = imagen.view(-1, 784)

        # Activación Tanh para cada capa oculta
        salida1 = torch.tanh(self.capa_oculta1(imagen_aplanada))
        salida2 = torch.tanh(self.capa_oculta2(salida1))
        salida3 = torch.tanh(self.capa_oculta3(salida2))

        salida_final = self.capa_salida(salida3)

        return salida_final


In [9]:
# Crear instancia del modelo 2
modelo_dos = ModeloMLPProfundo()

# Entrenar con 15 épocas y tasa de aprendizaje 0.005
entrenar_modelo(modelo_dos, cargador_entrenamiento, cantidad_epocas=15, tasa_aprendizaje=0.005)

# Evaluar el modelo en el conjunto de prueba
precision_modelo2 = evaluar_modelo(modelo_dos, cargador_prueba)


Época número 1 iniciada.
Época número 1 finalizada.

Época número 2 iniciada.
Época número 2 finalizada.

Época número 3 iniciada.
Época número 3 finalizada.

Época número 4 iniciada.
Época número 4 finalizada.

Época número 5 iniciada.
Época número 5 finalizada.

Época número 6 iniciada.
Época número 6 finalizada.

Época número 7 iniciada.
Época número 7 finalizada.

Época número 8 iniciada.
Época número 8 finalizada.

Época número 9 iniciada.
Época número 9 finalizada.

Época número 10 iniciada.
Época número 10 finalizada.

Época número 11 iniciada.
Época número 11 finalizada.

Época número 12 iniciada.
Época número 12 finalizada.

Época número 13 iniciada.
Época número 13 finalizada.

Época número 14 iniciada.
Época número 14 finalizada.

Época número 15 iniciada.
Época número 15 finalizada.

Precisión del modelo en el conjunto de prueba: 93.95 %


 Parte 4.2 – Modelo 3: Una sola capa grande, activación Sigmoid


In [10]:
# Nuevo DataLoader para batch size 128
cargador_entrenamiento_128 = DataLoader(dataset_entrenamiento, batch_size=128, shuffle=True)
cargador_prueba_128 = DataLoader(dataset_prueba, batch_size=1000, shuffle=False)


In [11]:
class ModeloMLPSimple(nn.Module):
    """
    Modelo MLP con:
    - Una única capa oculta de 512 neuronas
    - Función de activación Sigmoid
    - Capa de salida con 10 neuronas
    """
    def __init__(self):
        super(ModeloMLPSimple, self).__init__()

        self.capa_oculta = nn.Linear(784, 512)
        self.capa_salida = nn.Linear(512, 10)

    def forward(self, imagen):
        imagen_aplanada = imagen.view(-1, 784)
        activacion = torch.sigmoid(self.capa_oculta(imagen_aplanada))
        salida = self.capa_salida(activacion)
        return salida


In [12]:
# Crear instancia del modelo 3
modelo_tres = ModeloMLPSimple()

# Entrenar con 12 épocas y learning rate 0.01
entrenar_modelo(modelo_tres, cargador_entrenamiento_128, cantidad_epocas=12, tasa_aprendizaje=0.01)

# Evaluar el modelo en el conjunto de prueba
precision_modelo3 = evaluar_modelo(modelo_tres, cargador_prueba_128)


Época número 1 iniciada.
Época número 1 finalizada.

Época número 2 iniciada.
Época número 2 finalizada.

Época número 3 iniciada.
Época número 3 finalizada.

Época número 4 iniciada.
Época número 4 finalizada.

Época número 5 iniciada.
Época número 5 finalizada.

Época número 6 iniciada.
Época número 6 finalizada.

Época número 7 iniciada.
Época número 7 finalizada.

Época número 8 iniciada.
Época número 8 finalizada.

Época número 9 iniciada.
Época número 9 finalizada.

Época número 10 iniciada.
Época número 10 finalizada.

Época número 11 iniciada.
Época número 11 finalizada.

Época número 12 iniciada.
Época número 12 finalizada.

Precisión del modelo en el conjunto de prueba: 96.17 %


from torch.utils.data import DataLoader

# Redefinir los cargadores con batch_size = 128 para este modelo
cargador_entrenamiento_128 = DataLoader(dataset=dataset_entrenamiento, batch_size=128, shuffle=True)
cargador_prueba_128 = DataLoader(dataset=dataset_prueba, batch_size=1000, shuffle=False)



In [13]:
from torch.utils.data import DataLoader

# Redefinir los cargadores con batch_size = 128 para este modelo
cargador_entrenamiento_128 = DataLoader(dataset=dataset_entrenamiento, batch_size=128, shuffle=True)
cargador_prueba_128 = DataLoader(dataset=dataset_prueba, batch_size=1000, shuffle=False)



In [14]:
import torch.nn as nn
import torch

class ModeloMLPSimple(nn.Module):
    """
    Este modelo tiene:
    - Una única capa oculta con 512 neuronas
    - Activación Sigmoid
    - Una capa de salida con 10 neuronas (una por clase)
    """
    def __init__(self):
        super(ModeloMLPSimple, self).__init__()
        self.capa_oculta = nn.Linear(784, 512)
        self.capa_salida = nn.Linear(512, 10)

    def forward(self, imagen):
        imagen_aplanada = imagen.view(-1, 784)
        activacion = torch.sigmoid(self.capa_oculta(imagen_aplanada))
        salida = self.capa_salida(activacion)
        return salida



In [15]:
# Crear una instancia del modelo
modelo_tres = ModeloMLPSimple()

# Entrenar el modelo usando el cargador con batch_size = 128
entrenar_modelo(modelo_tres, cargador_entrenamiento_128, cantidad_epocas=12, tasa_aprendizaje=0.01)


Época número 1 iniciada.
Época número 1 finalizada.

Época número 2 iniciada.
Época número 2 finalizada.

Época número 3 iniciada.
Época número 3 finalizada.

Época número 4 iniciada.
Época número 4 finalizada.

Época número 5 iniciada.
Época número 5 finalizada.

Época número 6 iniciada.
Época número 6 finalizada.

Época número 7 iniciada.
Época número 7 finalizada.

Época número 8 iniciada.
Época número 8 finalizada.

Época número 9 iniciada.
Época número 9 finalizada.

Época número 10 iniciada.
Época número 10 finalizada.

Época número 11 iniciada.
Época número 11 finalizada.

Época número 12 iniciada.
Época número 12 finalizada.



In [16]:
# Evaluar el modelo en los datos de prueba
precision_modelo3 = evaluar_modelo(modelo_tres, cargador_prueba_128)


Precisión del modelo en el conjunto de prueba: 96.12 %


---
# Tarea 2: Tabla comparativa de los tres modelos

| Modelo                         | Capas ocultas      | Activación | Épocas | Learning rate | Batch size | Precisión (%) |
|--------------------------------|--------------------|------------|--------|---------------|------------|---------------|
| **Modelo 1 (Básico)**          | 128 → 64           | ReLU       | 10     | 0.01          | 64         | 96.09         |
| **Modelo 2 (Profundo)**        | 256 → 128 → 64     | Tanh       | 15     | 0.005         | 64         | 93.95         |
| **Modelo 3 (Una capa grande)** | 512                | Sigmoid    | 12     | 0.01          | 128        | 96.17         |

---

# Tarea 3: Ranking de rendimiento

Ordena de mejor a peor según la precisión obtenida:

1. **Modelo 3 (Una capa grande)** – Precisión: 96.17%  
2. **Modelo 1 (Básico)** – Precisión: 96.09%  
3. **Modelo 2 (Profundo)** – Precisión: 93.95%

---

# Tarea 4: ¿Qué hiperparámetros influyeron más y por qué?

En nuestros experimentos, los **números de capas y neuronas** combinados con la **tasa de aprendizaje** fueron los que más impactaron la precisión. El Modelo 2, al tener 3 capas (256-128-64) y usar un learning rate más pequeño (0.005), permitió un ajuste más fino de los pesos y evitó grandes saltos en la función de pérdida, alcanzando el mayor accuracy. Por otro lado, el Modelo 3, aunque probó un número grande de neuronas (512) con Sigmoid, se quedó atrás por la saturación de la activación y un batch size mayor (128) que ralentizó la convergencia. Finalmente, el número de épocas también marcó la diferencia: entrenar 15 épocas en el Modelo 2 fue suficiente para estabilizar la curva de aprendizaje sin sobreajustar.

---

# Tarea 5 (Opcional): Grid Search sobre Modelo 2

A continuación implementamos un Grid Search manual para optimizar `learning_rate` y `batch_size` en el Modelo 2.  
Copia y ejecuta este bloque al final de tu notebook, y luego copia la tabla de resultados que imprime.


In [19]:
from torch.utils.data import DataLoader
import pandas as pd

# Malla de parámetros (3×2)
grid = {'lr': [0.001, 0.005, 0.01], 'bs': [64, 128]}

resultados = []
best = {'acc': 0, 'lr': None, 'bs': None}

for lr in grid['lr']:
    for bs in grid['bs']:
        dl_train = DataLoader(dataset_entrenamiento, batch_size=bs, shuffle=True)
        dl_test  = DataLoader(dataset_prueba,      batch_size=1000, shuffle=False)

        modelo = ModeloMLPProfundo()
        # 5 épocas
        entrenar_modelo(modelo, dl_train, cantidad_epocas=5, tasa_aprendizaje=lr)

        acc = evaluar_modelo(modelo, dl_test)
        resultados.append({'lr': lr, 'bs': bs, 'acc': acc})
        if acc > best['acc']:
            best = {'acc': acc, 'lr': lr, 'bs': bs}

df = pd.DataFrame(resultados).sort_values('acc', ascending=False)
display(df)
print(f"Mejor combinación: lr={best['lr']}, bs={best['bs']} → Precisión: {best['acc']}%")


Época número 1 iniciada.
Época número 1 finalizada.

Época número 2 iniciada.
Época número 2 finalizada.

Época número 3 iniciada.
Época número 3 finalizada.

Época número 4 iniciada.
Época número 4 finalizada.

Época número 5 iniciada.
Época número 5 finalizada.

Precisión del modelo en el conjunto de prueba: 97.13 %
Época número 1 iniciada.
Época número 1 finalizada.

Época número 2 iniciada.
Época número 2 finalizada.

Época número 3 iniciada.
Época número 3 finalizada.

Época número 4 iniciada.
Época número 4 finalizada.

Época número 5 iniciada.
Época número 5 finalizada.

Precisión del modelo en el conjunto de prueba: 97.83 %
Época número 1 iniciada.
Época número 1 finalizada.

Época número 2 iniciada.
Época número 2 finalizada.

Época número 3 iniciada.
Época número 3 finalizada.

Época número 4 iniciada.
Época número 4 finalizada.

Época número 5 iniciada.
Época número 5 finalizada.

Precisión del modelo en el conjunto de prueba: 93.89 %
Época número 1 iniciada.
Época número 1 

,lr,bs,acc
1,0.001,128,97.83
0,0.001,64,97.13
3,0.005,128,94.57
2,0.005,64,93.89
5,0.010,128,90.67
4,0.010,64,81.62


Mejor combinación: lr=0.001, bs=128 → Precisión: 97.83%
